# Data Collection & Cleansing

# Dependencies

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
from pprint import pprint
from config import api_key
import numpy as np
import os
import shutil33q2

# Data Collection

In [2]:
#The College Scorecard API is a GET API that lives at http://api.data.gov/ed/collegescorecard/
#The endpoint for querying all data is /v1/schools
base_url = "https://api.data.gov/ed/collegescorecard/v1/schools?"
base_url = f"{base_url}api_key={api_key}&fields="
base_url

'https://api.data.gov/ed/collegescorecard/v1/schools?api_key=cwln6WtCqQN2nOVYktDIE6683aUseu0tPQ4JmJeZ&fields='

In [3]:
# List of all the search conditions
parameters = ["&school.degrees_awarded.predominant=3",
             "&school.institutional_characteristics.level=1",
             "&school.operating=1"
             ]

# Appending all the conditions values to construct the conditionss_url
parameters_url = ""
for parameter in parameters:
    parameters_url = parameters_url + parameter
parameters_url

'&school.degrees_awarded.predominant=3&school.institutional_characteristics.level=1&school.operating=1'

In [4]:
# Dictionary all the desired fields
year = "latest"
fields = {
          # School Category
          "School Name": "school.name",
          "School ID": "id",
          "School State": "school.state",
          "School Ownership": "school.ownership",
          "Full-time Faculty Rate (%)": "school.ft_faculty_rate",
          "Faculty's average salary per month": "school.faculty_salary",
          # Student Category
          "Student Enrollment Size": year + ".student.size",
          "Student Enrollment All": year + ".student.enrollment.all",
          "Male Students (%)": year + ".student.demographics.men",
          "Female Students (%)": year + ".student.demographics.women",
          "Retention Rate 4Yr (%)": year + ".student.retention_rate.four_year.full_time",
          #Cost Category
          "Attendance Cost per Academic Year": year + ".cost.attendance.academic_year",
          # Completion Category
          "150% Completion Rate at 4Yr (%)": year + ".completion.completion_rate_4yr_150nt",
          # Admissions Category 
          "Admission Rate (%)": year + ".admissions.admission_rate.overall",
          "SAT Average Overall": year + ".admissions.sat_scores.average.overall",
          "SAT 75th Percentile Critical Math": year + ".admissions.sat_scores.75th_percentile.math",
          "SAT 75th Percentile Critical Reading": year + ".admissions.sat_scores.75th_percentile.critical_reading",
          "SAT 75th Percentile Critical Writing": year + ".admissions.sat_scores.75th_percentile.writing",
          ## Earnings Category
          # 6 Years after Enrollment:
          "Mean Earnings (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.working_not_enrolled.mean_earnings",
          "Mean Male Earnings (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.mean_earnings.male_students",
          "Mean Female Earnings (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.mean_earnings.female_students",
          "Std. Deviation Earning (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.working_not_enrolled.std_dev",
          "Percent of Students Earning >$25K (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.percent_greater_than_25000",
          "Low Income Students (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.working_not_enrolled.income.lowest_tercile",
          "Medium Income Students (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.working_not_enrolled.income.middle_tercile",
          "High Income Students (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.working_not_enrolled.income.highest_tercile",
          "Mean Earnings Low (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.mean_earnings.lowest_tercile",
          "Mean Earnings Medium (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.mean_earnings.middle_tercile",
          "Mean Earnings High (6 Yrs after Entry)": year + ".earnings.6_yrs_after_entry.mean_earnings.highest_tercile",
          # 10 Years after Enrollment:
          "Mean Earnings (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.working_not_enrolled.mean_earnings",
          "Mean Male Earnings (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.mean_earnings.male_students",
          "Mean Female Earnings (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.mean_earnings.female_students",
          "Std. Deviation Earning (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.working_not_enrolled.std_dev",
          "Percent of Students Earning >$25K (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.percent_greater_than_25000",
          "Low Income Students (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.working_not_enrolled.income.lowest_tercile",
          "Medium Income Students (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.working_not_enrolled.income.middle_tercile",
          "High Income Students (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.working_not_enrolled.income.highest_tercile",
          "Mean Earnings Low (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.mean_earnings.lowest_tercile",
          "Mean Earnings Medium (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.mean_earnings.middle_tercile",
          "Mean Earnings High (10 Yrs after Entry)": year + ".earnings.10_yrs_after_entry.mean_earnings.highest_tercile"
         }

# Appending all the fields values to construct the fields_url
fields_url = ""
for key, val in fields.items():
    fields_url = fields_url + val + ","

# To remove the extra "," at the end of fields_url
fields_url = fields_url[:-1]
fields_url

'school.name,id,school.state,school.ownership,school.ft_faculty_rate,school.faculty_salary,latest.student.size,latest.student.enrollment.all,latest.student.demographics.men,latest.student.demographics.women,latest.student.retention_rate.four_year.full_time,latest.cost.attendance.academic_year,latest.completion.completion_rate_4yr_150nt,latest.admissions.admission_rate.overall,latest.admissions.sat_scores.average.overall,latest.admissions.sat_scores.75th_percentile.math,latest.admissions.sat_scores.75th_percentile.critical_reading,latest.admissions.sat_scores.75th_percentile.writing,latest.earnings.6_yrs_after_entry.working_not_enrolled.mean_earnings,latest.earnings.6_yrs_after_entry.mean_earnings.male_students,latest.earnings.6_yrs_after_entry.mean_earnings.female_students,latest.earnings.6_yrs_after_entry.working_not_enrolled.std_dev,latest.earnings.6_yrs_after_entry.percent_greater_than_25000,latest.earnings.6_yrs_after_entry.working_not_enrolled.income.lowest_tercile,latest.earnings

In [5]:
# Getting number of records returned to set the max page number
query_url = f"{base_url}{fields_url}{parameters_url}&page=0"
response = requests.get(query_url).json()
max_page_num = response["metadata"]["total"]//100 + 1
max_page_num

21

In [6]:
# Constructing the dataframe from the API request response

#Initializing variables
school_df = []
per_page = 100

for page_num in range(0,max_page_num):
    query_url = f"{base_url}{fields_url}{parameters_url}&page={page_num}&_per_page={per_page}"
    response = requests.get(query_url).json()
    
    for x in range(len(response["results"])):
        result_row = {}
        
        for key, val in fields.items(): 
            try:
                result_row[key] = response["results"][x][val]
            except KeyError:
                print(f"{key} key not found")
                
        school_df.append(result_row)

        
school_df = pd.DataFrame(school_df)

# Data Cleansing

In [7]:
# Updating School Ownership 1: "Public", 2: "Private NonProfit", 3: "Private ForProfit"}
school_df.loc[school_df["School Ownership"] == 1, "School Ownership"] = "Public"
school_df.loc[school_df["School Ownership"] == 2, "School Ownership"] = "Private NonProfit"
school_df.loc[school_df["School Ownership"] == 3, "School Ownership"] = "Private ForProfit"

In [8]:
# Formatting the Percentage columns
school_df["Admission Rate (%)"] = school_df["Admission Rate (%)"] * 100
school_df["150% Completion Rate at 4Yr (%)"] = school_df["150% Completion Rate at 4Yr (%)"] * 100
school_df["Retention Rate 4Yr (%)"] = school_df["Retention Rate 4Yr (%)"] * 100
school_df["Full-time Faculty Rate (%)"] = school_df["Full-time Faculty Rate (%)"] * 100
school_df["Percent of Students Earning >$25K (6 Yrs after Entry)"] = school_df["Percent of Students Earning >$25K (6 Yrs after Entry)"] * 100
school_df["Percent of Students Earning >$25K (10 Yrs after Entry)"] = school_df["Percent of Students Earning >$25K (10 Yrs after Entry)"] * 100
school_df["Male Students (%)"] = school_df["Male Students (%)"] * 100
school_df["Female Students (%)"] = school_df["Female Students (%)"] * 100

pd.options.display.float_format = '{:.2f}'.format

In [9]:
# Creating Directories & Saving Dataframe in CSV file
shutil.rmtree("Resources", ignore_errors = True)
shutil.rmtree("Plots", ignore_errors = True)

path_plot_ownadmission = os.path.join("Resources", "Plots", "Ownership_Admission")
os.makedirs(path_plot_ownadmission, exist_ok=True)

path_plot_select = os.path.join("Resources", "Plots", "Selectiveness")
os.makedirs(path_plot_select, exist_ok=True)

path_plot_compretention = os.path.join("Resources", "Plots", "Completion_Retention")
os.makedirs(path_plot_compretention, exist_ok=True)

path_plot_faculty = os.path.join("Resources", "Plots", "Faculty")
os.makedirs(path_plot_faculty, exist_ok=True)

path_csv_ownadmission = os.path.join("Resources", "CSV", "Ownership_Admission")
os.makedirs(path_csv_ownadmission, exist_ok=True)

path_csv_select = os.path.join("Resources", "CSV", "Selectiveness")
os.makedirs(path_csv_select, exist_ok=True)

path_csv_compretention = os.path.join("Resources", "CSV", "Completion_Retention")
os.makedirs(path_csv_compretention, exist_ok=True)

path_csv_faculty = os.path.join("Resources", "CSV", "Faculty")
os.makedirs(path_csv_faculty, exist_ok=True)

path_csv_general = os.path.join("Resources", "CSV", "General")
os.makedirs(path_csv_general, exist_ok=True)

# Saving Dataframe in CSV file

resource1 = "Schools_DF.csv"
school_df.to_csv(os.path.join(path_csv_general, resource1), index=False)